# Import library

In [1]:
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
import os 
import time
import mediapipe as mp

2023-08-08 21:28:13.466921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Make Keypoints using MP Holistic

In [16]:
mp_holistic = mp.solutions.holistic # Holistic Model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

NameError: name 'mp' is not defined

In [3]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    result = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, result

In [4]:
def draw_landmarks(image, result):
    mp_drawing.draw_landmarks(image, result.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def style_draw_landmark(image, result):
    mp_drawing.draw_landmarks(image, result.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [6]:
mp_holistic.FACEMESH_TESSELATION

frozenset({(18, 17),
           (82, 38),
           (8, 9),
           (456, 248),
           (167, 2),
           (303, 271),
           (69, 104),
           (253, 450),
           (41, 42),
           (315, 16),
           (73, 74),
           (339, 373),
           (258, 385),
           (105, 63),
           (219, 48),
           (304, 272),
           (236, 217),
           (231, 22),
           (85, 84),
           (365, 367),
           (237, 218),
           (243, 112),
           (436, 427),
           (409, 410),
           (248, 456),
           (330, 266),
           (136, 135),
           (88, 95),
           (394, 430),
           (107, 108),
           (118, 117),
           (439, 438),
           (442, 443),
           (260, 466),
           (115, 48),
           (342, 276),
           (121, 128),
           (200, 421),
           (113, 124),
           (132, 137),
           (23, 230),
           (343, 357),
           (443, 444),
           (62, 78),
           (26,

In [7]:
cap = cv.VideoCapture(0)

# set mediapie model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        #reade frame
        ret, frame = cap.read()
    
        #make detection
        image, result = mediapipe_detection(frame, holistic)
        print(result)
        
        #draw landmark
        style_draw_landmark(image, result)
        
        cv.imshow("Apa Ajalah", image)
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Extract Keypoints Values

In [8]:
result.pose_landmarks.landmark[0]

x: 0.49292725
y: 0.43770438
z: -0.4807838
visibility: 0.9999835

In [9]:
pose = []
for res in result.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [10]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(123)
lh = np.array([[res.x, res.y, res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)

In [11]:
def extract_keypoints(result):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, rh, lh])

In [12]:
extract_keypoints(result).shape

(1662,)

In [13]:
468*3+33*4+21*3+21*3

1662

# Setup Folders For Collections

In [17]:
# path for export data
Path = os.path.join('Dataset')

# actions detect
actions = np.array(['aku', 'terimakasih', 'siapa'])

# thirty video worth of data
no_squence = 30

# videos frame
squence_length = 30

In [18]:
for action in actions:
    for squence in range(no_squence):
        try:
            os.makedirs(os.path.join(Path, action, str(squence)))
        except:
            pass

# Collect Keypoints Values for Tranining and Testing

In [16]:
cap = cv.VideoCapture(0)

# set mediapie model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through action
    for action in actions:
        #Loop through squence aka videos
        for squence in range(no_squence):
            #Loop through video length aka squence length
            for frame_num in range(squence_length):
                
                #reade frame
                ret, frame = cap.read()

                #make detection
                image, result = mediapipe_detection(frame, holistic)
                print(result)

                #draw landmark
                style_draw_landmark(image, result)
                
                #New apply wait logic
                if frame_num == 0:
                    cv.putText(image, 'STARTING COLLECT', (120, 200),
                              cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv.LINE_AA)
                    cv.putText(image, 'Collecting frames for {} video number {}'.format(action, squence), (15, 12),
                              cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                    cv.imshow("Apa Ajalah", image)
                    cv.waitKey(2000)
                else:
                    cv.putText(image, 'Collecting frames for {} video number {}'.format(action, squence), (15, 12),
                              cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                
                    cv.imshow("Apa Ajalah", image)
                
                #Exteract Keypoints
                keypoints = extract_keypoints(result)
                npy_path = os.path.join(Path, action, str(squence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [17]:
cap.release()
cv.destroyAllWindows()

# Preprocessing Data And Create Labels Features

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2023-08-07 21:32:01.970817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
label_map = {label:num for num, label in enumerate(actions)}

In [21]:
label_map

{'aku': 0, 'terimakasih': 1, 'dimana': 2, 'kenapa': 3, 'nama': 4, 'siapa': 5}

In [22]:
squences, labels = [], []
for action in actions:
    for squence in range(no_squence):
        window = []
        for frame_num in range(squence_length):
            res = np.load(os.path.join(Path, action, str(squence), "{}.npy".format(frame_num)))
            window.append(res)
        squences.append(window)
        labels.append(label_map[action])

In [55]:
np.array(squences).shape

(180, 30, 1662)

In [56]:
np.array(labels).shape

(180,)

In [23]:
x = np.array(squences)

In [58]:
x.shape

(180, 30, 1662)

In [24]:
y = to_categorical(labels).astype(int)

In [60]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [26]:
print(x_train.shape, x_test.shape)

(171, 30, 1662) (9, 30, 1662)


# Build and Train LSTM Nueral Network

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [28]:
log_dir = os.path.join('Logs')
tg_callbacks = TensorBoard(log_dir=log_dir)


In [29]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [30]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.fit(x_train, y_train, epochs=1000, callbacks=[tg_callbacks])

Epoch 1/1000
6/6 [==============================] - 4s 81ms/step - loss: 1.8228 - categorical_accuracy: 0.1754
Epoch 2/1000
6/6 [==============================] - 0s 79ms/step - loss: 1.9891 - categorical_accuracy: 0.1930
Epoch 3/1000
6/6 [==============================] - 0s 72ms/step - loss: 1.8682 - categorical_accuracy: 0.1287
Epoch 4/1000
6/6 [==============================] - 0s 71ms/step - loss: 1.7548 - categorical_accuracy: 0.2105
Epoch 5/1000
6/6 [==============================] - 0s 74ms/step - loss: 1.9860 - categorical_accuracy: 0.1754
Epoch 6/1000
6/6 [==============================] - 0s 74ms/step - loss: 1.7626 - categorical_accuracy: 0.2515
Epoch 7/1000
6/6 [==============================] - 0s 72ms/step - loss: 1.7154 - categorical_accuracy: 0.2924
Epoch 8/1000
6/6 [==============================] - 1s 84ms/step - loss: 1.6683 - categorical_accuracy: 0.2339
Epoch 9/1000
6/6 [==============================] - 0s 79ms/step - loss: 1.6231 - categorical_accuracy: 0.3509
E

6/6 [==============================] - 0s 74ms/step - loss: 1.5850 - categorical_accuracy: 0.2924
Epoch 75/1000
6/6 [==============================] - 1s 215ms/step - loss: 1.5813 - categorical_accuracy: 0.2924
Epoch 76/1000
6/6 [==============================] - 1s 147ms/step - loss: 1.5758 - categorical_accuracy: 0.2924
Epoch 77/1000
6/6 [==============================] - 1s 74ms/step - loss: 1.5722 - categorical_accuracy: 0.2924
Epoch 78/1000
6/6 [==============================] - 1s 72ms/step - loss: 1.5680 - categorical_accuracy: 0.2924
Epoch 79/1000
6/6 [==============================] - 0s 67ms/step - loss: 1.5644 - categorical_accuracy: 0.2924
Epoch 80/1000
6/6 [==============================] - 0s 70ms/step - loss: 1.5612 - categorical_accuracy: 0.2924
Epoch 81/1000
6/6 [==============================] - 0s 67ms/step - loss: 1.5586 - categorical_accuracy: 0.2924
Epoch 82/1000
6/6 [==============================] - 1s 85ms/step - loss: 1.5561 - categorical_accuracy: 0.2924
Epoc

6/6 [==============================] - 1s 89ms/step - loss: 1.5202 - categorical_accuracy: 0.2749
Epoch 219/1000
6/6 [==============================] - 1s 85ms/step - loss: 1.5202 - categorical_accuracy: 0.2924
Epoch 220/1000
6/6 [==============================] - 0s 78ms/step - loss: 1.5203 - categorical_accuracy: 0.2924
Epoch 221/1000
6/6 [==============================] - 1s 87ms/step - loss: 1.5203 - categorical_accuracy: 0.2924
Epoch 222/1000
6/6 [==============================] - 1s 100ms/step - loss: 1.5204 - categorical_accuracy: 0.2982
Epoch 223/1000
6/6 [==============================] - 1s 104ms/step - loss: 1.5202 - categorical_accuracy: 0.3041
Epoch 224/1000
6/6 [==============================] - 1s 103ms/step - loss: 1.5203 - categorical_accuracy: 0.2924
Epoch 225/1000
6/6 [==============================] - 1s 103ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 226/1000
6/6 [==============================] - 1s 109ms/step - loss: 1.5202 - categorical_accuracy: 

6/6 [==============================] - 1s 107ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 363/1000
6/6 [==============================] - 1s 101ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 364/1000
6/6 [==============================] - 1s 128ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 365/1000
6/6 [==============================] - 1s 111ms/step - loss: 1.5198 - categorical_accuracy: 0.2924
Epoch 366/1000
6/6 [==============================] - 1s 119ms/step - loss: 1.5198 - categorical_accuracy: 0.2924
Epoch 367/1000
6/6 [==============================] - 1s 98ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 368/1000
6/6 [==============================] - 1s 95ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 369/1000
6/6 [==============================] - 1s 104ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 370/1000
6/6 [==============================] - 1s 104ms/step - loss: 1.5199 - categorical_accuracy

6/6 [==============================] - 1s 222ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 507/1000
6/6 [==============================] - 1s 153ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 508/1000
6/6 [==============================] - 1s 167ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 509/1000
6/6 [==============================] - 1s 186ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 510/1000
6/6 [==============================] - 1s 159ms/step - loss: 1.5200 - categorical_accuracy: 0.2573
Epoch 511/1000
6/6 [==============================] - 1s 91ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 512/1000
6/6 [==============================] - 1s 133ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 513/1000
6/6 [==============================] - 1s 111ms/step - loss: 1.5202 - categorical_accuracy: 0.2924
Epoch 514/1000
6/6 [==============================] - 1s 91ms/step - loss: 1.5200 - categorical_accuracy

6/6 [==============================] - 1s 94ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 651/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.5198 - categorical_accuracy: 0.2924
Epoch 652/1000
6/6 [==============================] - 1s 99ms/step - loss: 1.5198 - categorical_accuracy: 0.3041
Epoch 653/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 654/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 655/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.5202 - categorical_accuracy: 0.2924
Epoch 656/1000
6/6 [==============================] - 1s 97ms/step - loss: 1.5203 - categorical_accuracy: 0.2924
Epoch 657/1000
6/6 [==============================] - 1s 98ms/step - loss: 1.5205 - categorical_accuracy: 0.2924
Epoch 658/1000
6/6 [==============================] - 1s 93ms/step - loss: 1.5205 - categorical_accuracy: 0.292

6/6 [==============================] - 1s 125ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 795/1000
6/6 [==============================] - 1s 122ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 796/1000
6/6 [==============================] - 1s 232ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 797/1000
6/6 [==============================] - 1s 129ms/step - loss: 1.5199 - categorical_accuracy: 0.2982
Epoch 798/1000
6/6 [==============================] - 1s 181ms/step - loss: 1.5200 - categorical_accuracy: 0.2865
Epoch 799/1000
6/6 [==============================] - 1s 175ms/step - loss: 1.5203 - categorical_accuracy: 0.2398
Epoch 800/1000
6/6 [==============================] - 1s 130ms/step - loss: 1.5198 - categorical_accuracy: 0.2924
Epoch 801/1000
6/6 [==============================] - 1s 119ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 802/1000
6/6 [==============================] - 1s 134ms/step - loss: 1.5202 - categorical_accura

6/6 [==============================] - 1s 102ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 867/1000
6/6 [==============================] - 1s 99ms/step - loss: 1.5200 - categorical_accuracy: 0.2924
Epoch 868/1000
6/6 [==============================] - 1s 115ms/step - loss: 1.5199 - categorical_accuracy: 0.2924
Epoch 869/1000
6/6 [==============================] - 1s 124ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 870/1000
6/6 [==============================] - 1s 120ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 871/1000
6/6 [==============================] - 1s 86ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 872/1000
6/6 [==============================] - 1s 91ms/step - loss: 1.5202 - categorical_accuracy: 0.2924
Epoch 873/1000
6/6 [==============================] - 1s 91ms/step - loss: 1.5201 - categorical_accuracy: 0.2924
Epoch 874/1000
6/6 [==============================] - 1s 107ms/step - loss: 1.5199 - categorical_accuracy: 

In [68]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 32)                20608     
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 565,926
Trainable params: 565,926
Non-tr

# Make Prediction

In [69]:
model.predict(x_test)

1/1 [==============================] - 0s 447ms/step


array([[0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406596, 0.15703912, 0.16186146, 0.17016762, 0.17687921,
        0.16998662],
       [0.16406594, 0.15703909, 0.16186145, 0.17016761, 0.1768792 ,
        0.1699866 ]], dtype=float32)

In [70]:
actions[np.argmax(res[0])]

'aku'

In [71]:
actions[np.argmax(y_test[0])]

'dimana'

In [72]:
model.save('detection_17.h5')

In [41]:
del model

In [73]:
model.load_weights('detection_coba1.h5')

# Evaluate Model

In [74]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [75]:
yhat = model.predict(x_train)

6/6 [==============================] - 0s 27ms/step


In [76]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [77]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[143,   0],
        [ 28,   0]],

       [[144,   0],
        [ 27,   0]],

       [[143,   0],
        [ 28,   0]],

       [[142,   0],
        [ 29,   0]],

       [[  0, 141],
        [  0,  30]],

       [[142,   0],
        [ 29,   0]]])

In [78]:
accuracy_score(ytrue, yhat)

0.17543859649122806

# Testing

In [79]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv.putText(output_frame, actions[num], (0, 85+num*40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
    return output_frame

In [80]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        color = colors[num % len(colors)]  # Menggunakan indeks modulo
        cv.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), color, -1)
        cv.putText(output_frame, actions[num], (0, 85 + num * 40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)

    return output_frame

In [47]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

IndexError: list index out of range

<Figure size 1800x1800 with 0 Axes>

In [171]:
sequence.reverse()

NameError: name 'sequence' is not defined

In [81]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        style_draw_landmark(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv.putText(image, ' '.join(sentence), (3,30), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        
        # Show to screen
        cv.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
kenapa
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solu

1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
nama
<class 'mediapipe.python.soluti

In [ ]:
model.predict(x_test[0])